In [5]:
%cd ..

c:\Users\arik_\Documents\Dokumente\Job_Clausthal\TNTM\TNTM_Revision_TNNLS\TNTM


In [2]:
!pip install --upgrade gensim

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import pickle
import torch

from Code.Evaluate.Metrics import score_all, get_tw_embeddings
from Code.TNTM.TNTM_bow import TNTM_bow

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arik_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


current device: cpu
current device: cpu


In [8]:

#with open("C:\\Users\\arik_\\\Documents\\Dokumente\\Job_Clausthal\\TNTM\\TNTM_Revision_TNNLS\\TNTM\\Data\\DataOctis2\\octis_dataset_20ng.pickle", "rb") as file:
#    bow_data = pickle.load(file)

with open("C:\\Users\\arik_\\Documents\\Dokumente\\Job_Clausthal\\TNTM\\TNTM_Revision_TNNLS\\TNTM\Data\DataOctis2\\octis_dataset_20ng.pickle", "rb") as file:
    octis_dataset = pickle.load(file)

corpus = octis_dataset.get_corpus()

In [9]:
tw_emb = get_tw_embeddings(octis_dataset)

100%|██████████| 48018/48018 [10:08<00:00, 78.94it/s] 


In [10]:
with open("C:\\Users\\arik_\\Documents\\Dokumente\\Job_Clausthal\\TNTM\\TNTM_Revision_TNNLS\\TNTM\Data\DataOctis\\cleaned_embedding_df_20ng_BERT.pickle", 'rb') as f:
    embedding_df = pickle.load(f)

In [11]:
embedding_df.sort_values(by = "word", inplace = True)

In [12]:
embedding_ten_lis = []

for i in range(len(embedding_df)):
    embedding_ten_lis.append(embedding_df["embedding"].iloc[i])

In [13]:
embedding_df.sort_values(by = "word", inplace = True)
embedding_ten_lis = []

embedded_words = embedding_df.index.tolist()

for i in range(len(embedding_df)):
    embedding_ten_lis.append(embedding_df["embedding"].iloc[i])
embedding_ten = torch.stack(embedding_ten_lis)

In [14]:
vocab = embedded_words 
vocab_set = set(vocab)
corpus = [[word for word in doc if word in vocab_set] for doc in octis_dataset.get_corpus()]

In [24]:
model = TNTM_bow(
    n_topics = 10,
    save_path = "C:\\Users\\arik_\\Documents\\Dokumente\\Job_Clausthal\\TNTM\\TNTM_Revision_TNNLS\\TNTM\\msc\\SavedResults\\model_v1.pth",
    n_dims = 11,
    n_hidden_units = 200,
    n_encoder_layers = 3,
    enc_lr = 1e-4,
    dec_lr = 1e-3,
    n_epochs = 0,
    #batch_size = 128,
    batch_size = 256,
    dropout_rate_encoder = 0.3,
    prior_variance =  0.995, 
    prior_mean = None,
    n_topwords = 200,
    device = None, 
    validation_set_size = 0.2, 
    early_stopping = True,
    n_epochs_early_stopping = 10,
    return_embeddings = False,
    eps = 1e-4,
    umap_hyperparams = {'n_neighbors': 15, 'min_dist': 0.0},
    return_losses= True
)

In [25]:
res_all = model.fit(
    corpus = corpus, 
    vocab = vocab, 
    embeddings = embedding_ten
)

18846it [04:40, 67.22it/s] 
c:\Users\arik_\Documents\Dokumente\Job_Clausthal\TNTM\TNTM_Revision_TNNLS\TNTM\Code\TNTM\TNTM_bow.py:170: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mus_init_ten = torch.tensor(mus_init).to(self.device)
c:\Users\arik_\Documents\Dokumente\Job_Clausthal\TNTM\TNTM_Revision_TNNLS\TNTM\Code\TNTM\TNTM_bow.py:171: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  L_lower_init_ten = torch.tensor(L_lower_init).to(self.device)
c:\Users\arik_\Documents\Dokumente\Job_Clausthal\TNTM\TNTM_Revision_TNNLS\TNTM\Code\TNTM\TNTM_bow.py:172: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires

In [18]:
len(corpus)


18846

In [19]:
# remove all documents shorter than 10 words
corpus = [doc for doc in corpus if len(doc) >= 10]

In [20]:
octis_dataset._Dataset__corpus = corpus
octis_dataset._Dataset__vocab = vocab

In [21]:
evaluation_result = score_all(
    dataset = octis_dataset,
    tw_emb=tw_emb,
    n_words=10,
    result = {'topics': res_all[0], 
              "topic-word-matrix": res_all[1]},
)

100%|██████████| 5/5 [00:30<00:00,  6.06s/it]


In [26]:
res_all

(array([['tek', 'wustl', 'chem', ..., 'funny', 'german', 'less'],
        ['useless', 'simple', 'ideal', ..., 'domain', 'doc', 'zuma'],
        ['resident', 'party', 'leader', ..., 'locate', 'mailer', 'zuma'],
        ...,
        ['draft', 'pool', 'controller', ..., 'mark', 'mc', 'zuma'],
        ['simon', 'jay', 'jonathan', ..., 'fourth', 'france', 'zuma'],
        ['schedule', 'usage', 'address', ..., 'lewis', 'leo', 'zuma']],
       dtype='<U14'),
 array([[1.08505080e+11, 8.78742815e+10, 8.41016934e+10, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [5.59273120e+07, 4.95303320e+07, 4.81675920e+07, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [6.71369680e+07, 6.63744200e+07, 6.63236720e+07, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [4.86463120e+07, 4.77075920e+07, 3.71729280e+07, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.31885844e+10, 4.09234432e+10, 3.89777654

In [ ]:
def eval_model_n_topics(n_topics: int):
    model = TNTM_bow(
        n_topics = n_topics,
        save_path = "C:\\Users\\arik_\\Documents\\Dokumente\\Job_Clausthal\\TNTM\\TNTM_Revision_TNNLS\\TNTM\\msc\\SavedResults\\model_v1.pth",
        n_dims = 11,
        n_hidden_units = 200,
        n_encoder_layers = 3,
        enc_lr = 1e-4,
        dec_lr = 1e-3,
        n_epochs = 0,
        #batch_size = 128,
        batch_size = 256,
        dropout_rate_encoder = 0.3,
        prior_variance =  0.995, 
        prior_mean = None,
        n_topwords = 200,
        device = None, 
        validation_set_size = 0.2, 
        early_stopping = True,
        n_epochs_early_stopping = 10,
        return_embeddings = False,
        eps = 1e-4,
        umap_hyperparams = {'n_neighbors': 15, 'min_dist': 0.0},
        return_losses= True
    )

    res_all = model.fit(
        corpus = corpus, 
        vocab = vocab, 
        embeddings = embedding_ten
    )

    evaluation_result = score_all(
        dataset = octis_dataset,
        tw_emb=tw_emb,
        n_words=10,
        result = {'topics': res_all[0], 
                  "topic-word-matrix": res_all[1]},
    )
    
    return evaluation_result

In [ ]:
# evaluate different number of topics
from tqdm import tqdm

n_topics = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
n_iter = 10

results = {}

for m in range(n_iter):
    for n in tqdm(n_topics):
        results[n] = results.get(n, []) + [eval_model_n_topics(n)]

results


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

metrics = ['Embedding_Coherence', 'Topic Diversity', 'NPMI', "Perplextiy"]

# make a boxplot for each metric and each number of topics

for metric in metrics:
    data = []
    for n in n_topics:
        data.append([res[metric] for res in results[n]])
    plt.boxplot(data)
    plt.title(metric)
    plt.xticks(np.arange(len(n_topics))+1, n_topics)
    plt.show()


# save results
